In [1]:
import sys
sys.path.insert(1, '/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/codes/')

In [2]:
#del sys.modules["utilities"]
from utilities import *

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_master_index_all=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_all.pkl','rb'))

df_master_index_train=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_train.pkl','rb'))
df_master_index_val=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_val.pkl','rb'))
df_master_index_test=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_test.pkl','rb'))

df_master_index_train_val=pd.concat([df_master_index_train,df_master_index_val])

In [5]:
df_feats_clinical = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_clinical.pkl','rb'))
df_feats_icd_transform = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_icd_transform.pkl','rb'))
df_preor_ICULOS = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_preor_ICULOS.pkl','rb'))
df_feats_delirium = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_delirium.pkl','rb'))

Outcome_labels = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/Outcome_labels.pkl','rb'))

In [6]:
df_feats_clinical.shape,df_feats_icd_transform.shape,df_preor_ICULOS.shape,df_feats_delirium.shape

((136647, 2875), (136647, 1418), (136647, 4), (136647, 3))

In [7]:
df_feats_generalclinical = df_feats_clinical.merge(df_feats_icd_transform,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left').merge(df_preor_ICULOS,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left').merge(df_feats_delirium,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left')
                        

In [8]:
cutoff_y = np.quantile(Outcome_labels['LOS_postoperative'],0.75)
cutoff_y
Outcome_labels['PLOS'] =  (Outcome_labels['LOS_postoperative']>cutoff_y)*1
del Outcome_labels['LOS_postoperative']

Outcome_labels_train = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_val = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_test = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

len(Outcome_labels_train)+len(Outcome_labels_val)+len(Outcome_labels_test)-len(Outcome_labels)

0

In [9]:
df_feats_generalclinical_train = df_feats_generalclinical[df_feats_generalclinical['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

df_feats_generalclinical_val = df_feats_generalclinical[df_feats_generalclinical['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

df_feats_generalclinical_test = df_feats_generalclinical[df_feats_generalclinical['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

len(df_feats_generalclinical_train)+len(df_feats_generalclinical_val)+len(df_feats_generalclinical_test)-len(df_feats_generalclinical)

0

In [10]:
X_train = df_feats_generalclinical_train
X_val = df_feats_generalclinical_val
X_train_val = pd.concat([X_train,X_val])
X_test = df_feats_generalclinical_test

y_train = Outcome_labels_train
y_val = Outcome_labels_val
y_train_val = pd.concat([y_train,y_val])
y_test = Outcome_labels_test

In [11]:
X_train.shape,X_val.shape,X_train_val.shape,X_test.shape,y_train.shape,y_val.shape,y_train_val.shape,y_test.shape

((109318, 4297),
 (13665, 4297),
 (122983, 4297),
 (13664, 4297),
 (109318, 2),
 (13665, 2),
 (122983, 2),
 (13664, 2))

In [12]:
#experiment_baseline(X_train, X_val, y_train, y_val, seeds=[2023])

In [13]:
del sys.modules["utilities"]
from utilities import *

In [14]:
experiment_baseline(X_train_val, X_test, y_train_val, y_test, seeds=[666])

100%|██████████████████████████████████████| 1/1 [32:29:49<00:00, 116989.17s/it]


A. Scores on training set
avg XGBoost classifer: auc/  0.9713  F1:  0.8322  precision:  0.8154  recall:  0.8496  accuracy:  0.9158
std XGBoost classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
avg MLP classifer: auc/  0.9559  F1:  0.7962  precision:  0.7635  recall:  0.8318  accuracy:  0.8954
std MLP classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
avg logistic classifer: auc/  0.9238  F1:  0.7434  precision:  0.6699  recall:  0.8351  accuracy:  0.8584
std logistic classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
****************************************************

B. Scores on test set
avg XGBoost classifer: auc/  0.9652  F1:  0.8386  precision:  0.8435  recall:  0.8337  accuracy:  0.9074
std XGBoost classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
avg MLP classifer: auc/  0.9614  F1:  0.8333  precision:  0.824  recall:  0.8428  accuracy:  0.9027
std MLP classifer: auc/ 